In [1]:
import QUANTAXIS as QA
import datetime
import talib
import traceback
from finance_tools_py.backtest import BackTest
from finance_tools_py.backtest import AHundredChecker
import pandas as pd
import numpy as np
import concurrent.futures

filename="创业板.csv"

code_list=QA.QA_fetch_get_stock_list('pytdx')
## 合并所有股票一同回测
from tqdm import tqdm_notebook
print('Reading datas...')
datas={}
qfq_datas={}
hfq_datas={}
for symbol in tqdm_notebook(code_list['code'].values):
    try:
        if symbol[0]!="3":
            continue
        d=QA.QA_fetch_stock_day_adv(symbol)
        if d and not d.data.empty:
            qfq_datas[symbol]=d.to_qfq().data.reset_index().set_index('date')
            hfq_datas[symbol]=d.to_hfq().data.reset_index().set_index('date')
            datas[symbol]=d
    except:
        continue
print('Datas readed.')
code_list=datas.keys()

QUANTAXIS>> Selecting the Best Server IP of TDX


USING DEFAULT STOCK IP
USING DEFAULT FUTURE IP


QUANTAXIS>> === The BEST SERVER ===
 stock_ip 124.160.88.183 future_ip 124.74.236.94


Reading datas...


QA Error QA_fetch_stock_day_adv parameter code=300812 , start=1990-01-01, end=2020-03-09 call QA_fetch_stock_day return None
QA Error QA_fetch_stock_day_adv parameter code=300813 , start=1990-01-01, end=2020-03-09 call QA_fetch_stock_day return None
QA Error QA_fetch_stock_day_adv parameter code=300815 , start=1990-01-01, end=2020-03-09 call QA_fetch_stock_day return None
QA Error QA_fetch_stock_day_adv parameter code=300816 , start=1990-01-01, end=2020-03-09 call QA_fetch_stock_day return None
QA Error QA_fetch_stock_day_adv parameter code=300817 , start=1990-01-01, end=2020-03-09 call QA_fetch_stock_day return None
QA Error QA_fetch_stock_day_adv parameter code=300818 , start=1990-01-01, end=2020-03-09 call QA_fetch_stock_day return None
QA Error QA_fetch_stock_day_adv parameter code=300819 , start=1990-01-01, end=2020-03-09 call QA_fetch_stock_day return None
QA Error QA_fetch_stock_day_adv parameter code=300820 , start=1990-01-01, end=2020-03-09 call QA_fetch_stock_day return None


In [31]:

class chker(AHundredChecker):
    def __init__(self, buy_dict, sell_dict, min_price=3000, min_increase=1.15, **kwargs):
        super().__init__(buy_dict, sell_dict, **kwargs)
        self._min_price = 3000
        self._min_increase = min_increase

    def on_check_sell(self,
                      date: datetime.datetime.timestamp,
                      code: str,
                      price: float,
                      cash: float,
                      hold_amount: float,
                      hold_price: float) -> bool:
        if price < hold_price:
            # 当前价格小于持仓价时，不可卖
            return False
        if code in self.sell_dict.keys() and date in self.sell_dict[code]:
            # 其他情况均可卖
            return True
        if hold_amount > 0 and self._min_increase > 0 and price >= hold_price * self._min_increase:
            # 当前价格超过成本价15%时，可卖
            return True
        return False

    def on_calc_buy_amount(self, date, code: str, price: float, cash: float) -> float:
        amount = 100
        if self._min_price > 0:
            if cash < self._min_price:
                return super().on_calc_buy_amount(date, code, price, cash)
            while True:
                amount = amount + 100
                if price * amount > self._min_price:
                    break
            amount = amount - 100
        return amount

    def on_calc_sell_amount(self,
                            date: datetime.datetime.timestamp,
                            code: str,
                            price: float,
                            cash: float,
                            hold_amount: float,
                            hold_price: float) -> float:
        """返回所有持仓数量，一次卖出所有"""
        return hold_amount


def _calc_dict(df, symbol,start,end, bbands_periods, nbdevup, nbdevdn, difupstd, diflowstd, LINEARREG_ANGLE, MAX_LINEARREG_ANGLE,
               MAX_ROLLING_MEAN, MAX_ROLLING_MEAN_MIN_PERIODS):
    """
    
    Args:
        df (DataFrame): 前复权后的数据。reset_index().set_index('date')之后的数据。
    """
    if df is not None and not df.empty:
        bbands = df.reset_index().set_index('date')[start:end].reset_index().copy()
        bbands['up'], bbands['mean'], bbands['low'] = talib.BBANDS(bbands['close'].values, timeperiod=bbands_periods,
                                                                   nbdevup=nbdevup, nbdevdn=nbdevdn)
        bbands['diff_up'] = bbands['up'] / bbands['close']
        bbands['diff_mean'] = bbands['mean'] / bbands['close']
        bbands['diff_low'] = bbands['low'] / bbands['close']

        bbands['opt'] = 0

        m1 = bbands['diff_up'].min() + bbands['diff_up'].std() * difupstd  # 卖出点
        m2 = bbands['diff_low'].max() - bbands['diff_low'].std() * diflowstd  # 买入点

        angles = []
        max_angles = {}
        for angle in LINEARREG_ANGLE:
            col = 'angle_{}'.format(angle)
            angles.append(col)
            bbands[col] = talib.LINEARREG_ANGLE(bbands['close'], angle).abs().fillna(99)  # 日均线角度
            if MAX_LINEARREG_ANGLE == "MEAN":
                v = talib.LINEARREG_ANGLE(bbands['close'], angle).abs().mean()  # 日均线角度平均值
            else:
                v = talib.LINEARREG_ANGLE(bbands['close'], angle).abs().median()  # 日均线角度中位数
            if np.isnan(v):
                raise ValueError('{} LINEARREG_ANGLE is NaN'.format(angle))
            max_angles[col] = int(v)
        #                 bbands['angle_3']=talib.LINEARREG_ANGLE(bbands['close'],3).abs().fillna(99)#3日均线角度
        #                 max_angle_60=int(talib.LINEARREG_ANGLE(bbands['close'],60).abs().mean())#60日均线角度中位数

        bbands['rolling_mean'] = bbands['close'].rolling(MAX_ROLLING_MEAN,
                                                         min_periods=MAX_ROLLING_MEAN_MIN_PERIODS).mean()  # 250日均价
        bbands['rolling_std'] = bbands['close'].rolling(MAX_ROLLING_MEAN,
                                                        min_periods=MAX_ROLLING_MEAN_MIN_PERIODS).std()  # 250日均价
        bbands['rolling_mean'] = bbands['rolling_mean'] + bbands['rolling_std']

        """
        1. 股价偏离30日均线上限或下线(diff_up,diff_low)
        2. 30日均线角度绝对值小于5度时
        3. 3日均线角度绝对值小于10度时
        4. 当前价格需要小于250日均价+标准差的上限

        2~4 均是为了防止站在山顶上
        """

        bbands.loc[bbands['diff_up'] < m1, 'opt'] = 1  # 卖出点
        # 买入点
        query = 'diff_low>{}'.format(m2)
        query = query + ' & close<rolling_mean & '
        query = query + (' & '.join(['{}<{}'.format(a, max_angles[a]) for a in angles]))
        #             queries[symbol]=query
        bbands.loc[bbands.query(query).index, 'opt'] = 2
        bbands.loc[bbands['opt'] != 0][['date', 'close', 'opt']]
        point = bbands.loc[bbands['opt'] != 0][1:]

        return symbol, pd.DatetimeIndex(point.loc[point['opt'] == 2]['date'].values), pd.DatetimeIndex(point.loc[point['opt'] == 1]['date'].values), query
    return symbol,pd.Series(),pd.Series(),None

def calc_dict(qfq_datas: dict,
              start='2006-01-01',
              end='2018-12-31',
              bbands_periods=30,
              nbdevup=2.8,
              nbdevdn=2.8,
              difupstd=0.1,
              diflowstd=0.1,
              LINEARREG_ANGLE=[60, 30, 10, 5, 3],
              MAX_LINEARREG_ANGLE='MED',
              MAX_ROLLING_MEAN=250,
              MAX_ROLLING_MEAN_MIN_PERIODS=30,
              max_workers=None,
              mulitThread=False
              ):
    """计算买入卖出时机点

    Args:
        MAX_LINEARREG_ANGLE: MED（中位数），MEAN（平均值）

    Returns:
        {},{} : 买入时机字典，卖出时机字典
    """

    buy_dict = {}
    sell_dict = {}
    queries = {}
    if mulitThread:
#         with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
#             # Start the load operations and mark each future with its URL
#             _calcs = {executor.submit(_calc_dict, df, symbol,start,end, bbands_periods, nbdevup, nbdevdn, difupstd, diflowstd,
#                                       LINEARREG_ANGLE,
#                                       MAX_LINEARREG_ANGLE,
#                                       MAX_ROLLING_MEAN,
#                                       MAX_ROLLING_MEAN_MIN_PERIODS): (symbol, df) for (symbol, df) in qfq_datas.items()}
#             for future in concurrent.futures.as_completed(_calcs):
#                 try:
#                     symbol, buy, sell, q =future.result()
#                     if (not buy.empty or not sell.empty) and q:
#                         buy_dict[symbol] = buy
#                         sell_dict[symbol] = sell
#                         queries[symbol] = q
#                 except Exception as e:
#                     #                 print(str(e))
#                     #                 traceback.print_exc()
#                     if symbol in buy_dict:
#                         del buy_dict[symbol]
#                     if symbol in sell_dict:
#                         del sell_dict[symbol]
#                     if symbol in queries:
#                         del queries[symbol]
#                     continue
        pass
    else:
        for symbol,df in tqdm_notebook(qfq_datas.items()):
            try:
                symbol, buy_dict[symbol],sell_dict[symbol],queries[symbol]=_calc_dict(df, symbol,start,end, bbands_periods, nbdevup, nbdevdn, difupstd, diflowstd,
                                      LINEARREG_ANGLE,
                                      MAX_LINEARREG_ANGLE,
                                      MAX_ROLLING_MEAN,
                                      MAX_ROLLING_MEAN_MIN_PERIODS)
            except Exception as e:
                if symbol in buy_dict:
                    del buy_dict[symbol]
                if symbol in sell_dict:
                    del sell_dict[symbol]
                if symbol in queries:
                    del queries[symbol]
                continue
    return buy_dict, sell_dict, queries


def save_test(bt,start,end,min_price,min_increase,bbands_periods,nbdevup,nbdevdn,difupstd,diflowstd,MAX_LINEARREG_ANGLE,MAX_ROLLING_MEAN,MAX_ROLLING_MEAN_MIN_PERIODS):
    
    df=pd.DataFrame(
        {
            'available_cash':[np.round(bt.available_cash,2)],
            'total_assets_cur':[np.round(bt.total_assets_cur,2)],
            'trade_times':[len(bt.history_df)],
            'init_cash':[bt.init_cash],
            'start':[start],
            'end':[end],
            'bbands_periods':[bbands_periods],
            'nbdevup':[nbdevup],
            'nbdevdn':[nbdevdn],
            'difupstd':[difupstd],
            'diflowstd':[diflowstd],
            'MAX_LINEARREG_ANGLE':[MAX_LINEARREG_ANGLE],
            'MAX_ROLLING_MEAN':[MAX_ROLLING_MEAN],
            'MAX_ROLLING_MEAN_MIN_PERIODS':[MAX_ROLLING_MEAN_MIN_PERIODS],
            'min_price':[min_price],
            'min_increase':[min_increase],
        })
#     print(df[['start','end','min_price','available_cash','total_assets_cur','trade_times']])
    
    import os
    if os.path.exists(filename):
        df=pd.read_csv(filename,index_col=False).append(df)
    df.to_csv(filename,index=False)
        
def test(datas,
         qfq,
         hfq,
         start='2006-01-01',
         end='2018-12-31',
         bbands_periods=30,
         nbdevup=2.8,
         nbdevdn=2.8,
         difupstd=0.1,
         diflowstd=0.1,
         LINEARREG_ANGLE=[60, 30, 3],
         MAX_LINEARREG_ANGLE='MED',
         MAX_ROLLING_MEAN=250,
         MAX_ROLLING_MEAN_MIN_PERIODS=30,
         init_cash=50000,
         min_price=3000,
         min_increase=1.15,
         name='',max_workers=None):
    print(
        'start={},end={},bbands_periods={},nbdevup={},nbdevdn={},diffupstd={},diflowstd={},MAX_LINEARREG_ANGLE={},MAX_ROLLING_MEAN={},MAX_ROLLING_MEAN_MIN_PERIODS={},init_cash={},min_price={},min_increase={}'.format(
            start,end,bbands_periods, nbdevup, nbdevdn, difupstd, diflowstd, MAX_LINEARREG_ANGLE, MAX_ROLLING_MEAN,
            MAX_ROLLING_MEAN_MIN_PERIODS, init_cash, min_price, min_increase))
    buy_dict, sell_dict, queries = calc_dict(qfq,
                                             start=start,
                                             end=end,
                                             bbands_periods=bbands_periods,
                                             nbdevup=nbdevup,
                                             nbdevdn=nbdevdn,
                                             difupstd=difupstd,
                                             diflowstd=diflowstd,
                                             LINEARREG_ANGLE=LINEARREG_ANGLE,
                                             MAX_LINEARREG_ANGLE=MAX_LINEARREG_ANGLE,
                                             MAX_ROLLING_MEAN=MAX_ROLLING_MEAN,
                                             MAX_ROLLING_MEAN_MIN_PERIODS=MAX_ROLLING_MEAN_MIN_PERIODS,
                                             max_workers=max_workers
                                             )

    buy_symbols = [k for k, v in buy_dict.items() if v is not None and len(v) > 0]

    if len(buy_symbols) <= 0:
        print('没有买入信号')
        return None, None

#     print('Data reading...')
    datas_hfq = {}
#     for symbol in tqdm_notebook(buy_symbols):
    for symbol in buy_symbols:
        datas_hfq[symbol] = hfq[symbol][_end:]
        #从判断时间开始后开始计算
#     print('Data readed')

    all_data = pd.concat([x for x in datas_hfq.values() if x is not None])
    bt = BackTest(all_data.sort_index(level=0).reset_index(), init_cash,
                  callbacks=[chker(buy_dict, sell_dict, min_price, min_increase)])
    print('Calcing...{}'.format(len(buy_symbols)))
    bt.calc_trade_history(verbose=0)
    save_test(bt,start,end,min_price,min_increase,bbands_periods,nbdevup,nbdevdn,difupstd,diflowstd,MAX_LINEARREG_ANGLE,MAX_ROLLING_MEAN,MAX_ROLLING_MEAN_MIN_PERIODS)
    return bt, queries

In [32]:
result={}
queries={}
max_workers=None

In [ ]:
for _start in ['2006-01-01']:
    for _end in ['2018-01-01']:
        for _rollin_mean in [120,250,360]:
            for _bbands_periods in [5,20,30,60]:#布林带窗口期
                for _nbdev in np.arange(2.6,3.1,0.1):#布林带上下限大小
                    for _nb_diff in np.arange(0.1,1.0,0.2):#布林带上下限加减的标准差大小
                        for _min_price in [1000,3000,5000]:#最小购买金额
                            for _angle in ['MED','MEAN']:
                                name='bbands_periods={},nbdevup={},nbdevdn={},diffupstd={},diflowstd={},MAX_LINEARREG_ANGLE={},MAX_ROLLING_MEAN={},MAX_ROLLING_MEAN_MIN_PERIODS={},min_price={}'.format(
                                    _bbands_periods,_nbdev,_nbdev,_nb_diff,_nb_diff,_angle,_rollin_mean,_bbands_periods,_min_price)
                                b,q=test(datas,qfq_datas,hfq_datas,
                                         start=_start,end=_end,
                                                  name=name,
                                                  min_price=_min_price,
                                                  MAX_LINEARREG_ANGLE=_angle,
                                                  nbdevup=_nbdev,
                                                  nbdevdn=_nbdev,
                                                  difupstd=_nb_diff,
                                                  diflowstd=_nb_diff,
                                                  bbands_periods=_bbands_periods,
                                                  MAX_ROLLING_MEAN_MIN_PERIODS=_bbands_periods,
                                                  MAX_ROLLING_MEAN=_rollin_mean,
                                         max_workers=max_workers
                                                 )
                                result[name]=b
                                queries[name]=q

start=2006-01-01,end=2018-01-01,bbands_periods=5,nbdevup=2.6,nbdevdn=2.6,diffupstd=0.1,diflowstd=0.1,MAX_LINEARREG_ANGLE=MED,MAX_ROLLING_MEAN=120,MAX_ROLLING_MEAN_MIN_PERIODS=5,init_cash=50000,min_price=1000,min_increase=1.15


Calcing...450


start=2006-01-01,end=2018-01-01,bbands_periods=5,nbdevup=2.6,nbdevdn=2.6,diffupstd=0.1,diflowstd=0.1,MAX_LINEARREG_ANGLE=MEAN,MAX_ROLLING_MEAN=120,MAX_ROLLING_MEAN_MIN_PERIODS=5,init_cash=50000,min_price=1000,min_increase=1.15


Calcing...603


start=2006-01-01,end=2018-01-01,bbands_periods=5,nbdevup=2.6,nbdevdn=2.6,diffupstd=0.1,diflowstd=0.1,MAX_LINEARREG_ANGLE=MED,MAX_ROLLING_MEAN=120,MAX_ROLLING_MEAN_MIN_PERIODS=5,init_cash=50000,min_price=3000,min_increase=1.15


...

start=2006-01-01,end=2018-01-01,bbands_periods=5,nbdevup=2.6,nbdevdn=2.6,diffupstd=0.7000000000000001,diflowstd=0.7000000000000001,MAX_LINEARREG_ANGLE=MEAN,MAX_ROLLING_MEAN=120,MAX_ROLLING_MEAN_MIN_PERIODS=5,init_cash=50000,min_price=3000,min_increase=1.15


Calcing...638


start=2006-01-01,end=2018-01-01,bbands_periods=5,nbdevup=2.6,nbdevdn=2.6,diffupstd=0.7000000000000001,diflowstd=0.7000000000000001,MAX_LINEARREG_ANGLE=MED,MAX_ROLLING_MEAN=120,MAX_ROLLING_MEAN_MIN_PERIODS=5,init_cash=50000,min_price=5000,min_increase=1.15


Calcing...512


start=2006-01-01,end=2018-01-01,bbands_periods=5,nbdevup=2.6,nbdevdn=2.6,diffupstd=0.7000000000000001,diflowstd=0.7000000000000001,MAX_LINEARREG_ANGLE=MEAN,MAX_ROLLING_MEAN=120,MAX_ROLLING_MEAN_MIN_PERIODS=5,init_cash=50000,min_price=5000,min_increase=1.15
